In [1]:
import numpy as np
import torch
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from icp import ICP_ROT
from nc_score import pose_err
from tqdm import tqdm
import tools
import rpmg
import argparse
from dataset.CameraPoseDataset import CameraPoseDatasetPred
from rpr import find_poses

In [2]:
dataset_path = '/home/runyi/Data/7Scenes'
cal_labels_file = '/home/runyi/Project/TBCP6D/dataset/7Scenes_0.5/abs_7scenes_pose.csv_chess_cal.csv_results.csv'
test_labels_file = '/home/runyi/Project/TBCP6D/dataset/7Scenes_0.5/abs_7scenes_pose.csv_chess_test.csv_results.csv'
cal_set = CameraPoseDatasetPred(dataset_path, cal_labels_file)
test_set = CameraPoseDatasetPred(dataset_path, test_labels_file)

In [3]:
cal_set.poses.shape, cal_set.pred_poses.shape

((2417, 7), (2417, 7))

In [4]:
icp_rot = ICP_ROT(torch.tensor(cal_set.poses[:, 3:]), torch.tensor(cal_set.pred_poses[:, 3:]))

In [5]:
dataloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=1)

In [30]:
for idx, minibatch in enumerate(dataloader):
    img = minibatch.get('img')[0].numpy()
    pred_pose = minibatch.get('est_pose')
    pred_R = tools.compute_rotation_matrix_from_quaternion(pred_pose[:, 3:])
    pred_t = pred_pose[:, :3]
    # Get pred pose from R and t
    pred_pose_ = np.zeros((4,4))
    pred_pose_[:3, :3] = pred_R[0]
    pred_pose_[:3, 3] = pred_t[0]
    pred_pose_[3, 3] = 1
    for cal_img in tqdm(cal_set.imgs):
        relative_pose = find_poses(img, cal_img)
        print(relative_pose.shape, pred_pose_.shape)
        cal_dis_pose = pred_pose_.T @ relative_pose
        print(cal_dis_pose)

  0%|          | 0/2417 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'shape'

In [7]:
cal_set.img_paths[0]

'/home/runyi/Data/7Scenes/chess/seq-01/frame-000000.color.png'

In [8]:
test_set.img_paths[0]

'/home/runyi/Data/7Scenes/chess/seq-03/frame-000000.color.png'